### This was run on colab

In [23]:
from google.colab import drive
import pandas as pd
import torch

In [22]:
from transformers import BertTokenizer, BertModel #kills kernel if run with others

In [11]:
# Mount Google Drive
drive.mount('/content/gdrive')

# Define the file path to the just cleaned df
# (I also put them here: https://drive.google.com/drive/folders/1lXVVxWaR-fJ4htHx8J5C2HtoPSiTM_BX if you want to bind there)
path = "/content/gdrive/MyDrive/AdvancedML/final_project_data/metadata_w_2020articles_cleaned.csv"

df = pd.read_csv(path,  index_col=0)

# TODO: also load the lemmatized data and see how that

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [20]:
test_article = df.loc[df.loc[:, 'uuid'] == 'bcbc6bb2-406e-11ee-a96e-33dec8f414a2', :]
test_text = test_article['title_text'][0]
test_text

'trump shifts tone, says he’s ‘proud’ of fed amid coronavirus turmoil president trump on monday said he was “proud” of federal reserve chairman jerome powell’s handling of the economic turmoil brought on by the coronavirus pandemic, backing down from more than a year of criticizing his hand-picked fed chief and threatening his job security.trump praised powell during a monday evening briefing at the white house after the fed that morning drastically expanded its efforts to protect the u.s. economy and financ'

In [8]:
# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [21]:
encoding = tokenizer.encode_plus(test_text, add_special_tokens = True,
                                 truncation = True, return_tensors = "pt")

# Print the tokens
print("Original Text:", test_text)
print("Tokenized Text:", encoding)

Original Text: trump shifts tone, says he’s ‘proud’ of fed amid coronavirus turmoil president trump on monday said he was “proud” of federal reserve chairman jerome powell’s handling of the economic turmoil brought on by the coronavirus pandemic, backing down from more than a year of criticizing his hand-picked fed chief and threatening his job security.trump praised powell during a monday evening briefing at the white house after the fed that morning drastically expanded its efforts to protect the u.s. economy and financ
Tokenized Text: {'input_ids': tensor([[  101,  8398, 12363,  4309,  1010,  2758,  2002,  1521,  1055,  1520,
          7098,  1521,  1997,  7349, 13463, 21887, 23350, 17930,  2343,  8398,
          2006,  6928,  2056,  2002,  2001,  1523,  7098,  1524,  1997,  2976,
          3914,  3472, 11120,  8997,  1521,  1055,  8304,  1997,  1996,  3171,
         17930,  2716,  2006,  2011,  1996, 21887, 23350,  6090,  3207,  7712,
          1010,  5150,  2091,  2013,  2062,  20

In [24]:
model = BertModel.from_pretrained('bert-base-uncased')

def get_cls_sentence(sentence):
    # Tokenize input sentence and convert to tensor
    input_ids = torch.tensor([tokenizer.encode(sentence, add_special_tokens=True, max_length=512)])

    # Pass input through BERT model and extract embeddings for [CLS] token
    with torch.no_grad():
        outputs = model(input_ids)
        cls_embedding = outputs[0][:, 0, :]

    return cls_embedding.flatten()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [25]:
test_article['bert_sent_embedding'] = test_article['title_text'].apply(lambda sentence: get_cls_sentence(sentence))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
<ipython-input-25-8f1bea275e8b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_article['bert_sent_embedding'] = test_article['title_text'].apply(lambda sentence: get_cls_sentence(sentence))


In [ ]:
# starting with title only embeddings
df['bert_sentence_embedding_title_only'] = df['title'].apply(lambda sentence: get_cls_sentence(sentence))

In [ ]:
# pickling so that we don't have to rerun the embedding process every time
df.to_pickle("/content/gdrive/MyDrive/AdvancedML/final_project_data/clean_2020articles_w_title_embeddings.pkl")